sklearn 라이브러리를 이용해서 원핫인코딩을 편하게 처리할 수 있다.<br>
데이터프레임 df의 'hp_bin' 열에 들어 있는 범주형 데이터를 0, 1을 원소로 갖는 원핫벡터로 변환한다.<br>
결과는 선형대수학에서 정의하는 희소행렬(sparse matrix)로 정리된다.<br>
> *희소 행렬 (sparse matrix)
> > 행렬을 구성하는 원소들의 대부분이 0 값을 가지는 2차원 행렬.<br>행렬의 전체 크기에 비하여 실제 유용한 정보들은 일부만 존재하기 때문에 별도의 작업 처리를 사용하여 보다 효율적으로 적은 기억 장소에 표현하는 방법들이 사용되고 있다.

예제에서는 1차원 벡터를 2차원 행렬로 변환하고 다시 희소행렬로 변환한다.<br>
희소행렬은 (행, 열) 좌표와 값 형태로 정리된다. 앞에서 (0, 1)은 0행의 1열 위치를 말하고, 데이터 값은 숫자 1.0이 입력된다.

In [2]:
import pandas as pd
import numpy as np

# read_csv() 함수로 df 생성
df = pd.read_csv('./auto-mpg.csv', header=None)

# 열 이름 지정
df.columns = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight',
              'acceleration', 'model year', 'origin', 'name']

# horsepower 열의 누락 데이터('?')를 삭제하고 실수형으로 변환
df['horsepower'].replace('?', np.nan, inplace=True)     # '?'을 np.nan으로 변경
df.dropna(subset=['horsepower'], axis=0, inplace=True)  # 누락 데이터 행 삭제
df['horsepower'] = df['horsepower'].astype('float')

# np.histogram 함수로 3개의 bin으로 구분할 경계값의 리스트 구하기
count, bin_dividers = np.histogram(df['horsepower'], bins=3)

# 3개의 bin에 이름 지정
bin_names = ['저출력', '보통출력', '고출력']

# pd.cut 함수로 각 데이터를 3개의 bin에 할당
df['hp_bin'] = pd.cut(x=df['horsepower'],       # 데이터 배열
                      bins=bin_dividers,        # 경계값 리스트
                      labels=bin_names,         # bin 이름
                      include_lowest=True)      # 첫 경계값 포함

# hp_bin 열의 범주형 데이터를 더미 변수로 변환
horsepower_dummies = pd.get_dummies(df['hp_bin'])

# -- 여기까지 예제 5-11과 동일 --

# sklearn 라이브러리 불러오기
from sklearn import preprocessing

# 전처리를 위한 encoder 객체 만들기
label_encoder = preprocessing.LabelEncoder()        # label encoder 생성
onehot_encoder = preprocessing.OneHotEncoder()      # one hot encoder 생성

# label encoder로 문자열 범주를 숫자형 범주로 변환
onehot_labeled = label_encoder.fit_transform(df['hp_bin'].head(15))
print(onehot_labeled)
print(type(onehot_labeled))

# 2차원 행렬로 변경
onehot_reshaped = onehot_labeled.reshape(len(onehot_labeled), 1)
print(onehot_reshaped)
print(type(onehot_reshaped))

# 희소행렬로 변환
onehot_fitted = onehot_encoder.fit_transform(onehot_reshaped)
print(onehot_fitted)
print(type(onehot_fitted))


[1 1 1 1 1 0 0 0 0 0 0 1 1 0 2]
<class 'numpy.ndarray'>
[[1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [2]]
<class 'numpy.ndarray'>
  (0, 1)	1.0
  (1, 1)	1.0
  (2, 1)	1.0
  (3, 1)	1.0
  (4, 1)	1.0
  (5, 0)	1.0
  (6, 0)	1.0
  (7, 0)	1.0
  (8, 0)	1.0
  (9, 0)	1.0
  (10, 0)	1.0
  (11, 1)	1.0
  (12, 1)	1.0
  (13, 0)	1.0
  (14, 2)	1.0
<class 'scipy.sparse.csr.csr_matrix'>
